In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer


In [2]:
steam_data = pd.read_csv('../data/steam.csv')
steam_data.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [8]:
steam_data_filtered = steam_data[steam_data['positive_ratings'] > 1000]
steam_data_filtered.reset_index(drop=True, inplace=True)
steam_data_filtered.describe()

,appid,english,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,price
count,2.213000e+03,2213.000000,2213.000000,2213.000000,2.213000e+03,2213.000000,2213.000000,2213.000000,2213.000000
mean,3.433830e+05,0.995029,1.898328,44.059196,1.127571e+04,2217.720741,978.653864,773.679169,11.722878
std,2.106236e+05,0.070343,5.376516,128.261493,6.555857e+04,14840.940577,3069.256534,2752.327156,10.746110
min,1.000000e+01,0.000000,0.000000,0.000000,1.003000e+03,19.000000,0.000000,0.000000,0.000000
25%,2.282000e+05,1.000000,0.000000,0.000000,1.587000e+03,256.000000,128.000000,118.000000,3.990000
50%,3.181300e+05,1.000000,0.000000,26.000000,2.898000e+03,560.000000,293.000000,276.000000,9.990000
75%,4.598200e+05,1.000000,0.000000,50.000000,7.173000e+03,1376.000000,773.000000,650.000000,15.490000
max,1.046030e+06,1.000000,18.000000,3000.000000,2.644404e+06,487076.000000,63481.000000,63490.000000,60.990000


In [7]:
steam_data_filtered.shape

(2213, 18)

In [9]:
columns =['steamspy_tags','name']

steam_data_filtered[columns].isnull().values.any()

False

In [10]:
#Combine the values of the important columns into a single string
def get_important_features(data):
    important_features =[]
    for i in range(0,data.shape[0]):
        important_features.append(data['steamspy_tags'][i]+' '+data['name'][i])
        
    return important_features

In [12]:
steam_data_filtered['important_features']=get_important_features(steam_data_filtered)
steam_data_filtered['Game_no'] = range(0,steam_data_filtered.shape[0])
#Show the data
steam_data_filtered.head()

C:\Users\huang\AppData\Local\Temp\ipykernel_4260\3965404439.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_data_filtered['important_features']=get_important_features(steam_data_filtered)
C:\Users\huang\AppData\Local\Temp\ipykernel_4260\3965404439.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_data_filtered['Game_no'] = range(0,steam_data_filtered.shape[0])


,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,important_features,Game_no
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19,Action;FPS;Multiplayer Counter-Strike,0
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99,Action;FPS;Multiplayer Team Fortress Classic,1
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99,FPS;World War II;Multiplayer Day of Defeat,2
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99,Action;FPS;Multiplayer Deathmatch Classic,3
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99,FPS;Action;Sci-fi Half-Life: Opposing Force,4


In [36]:
cm = CountVectorizer().fit_transform(steam_data_filtered['important_features'])
#Get the cosine similarity matrix from cm
cs = cosine_similarity(cm)


In [37]:
#Get the title the user likes
title = 'Counter-Strike'

#Find the appid
app_id = steam_data_filtered[steam_data_filtered.name == title]['Game_no'].values[0]
print(app_id)

0


In [38]:
#Create a list of similarity score
scores = list(enumerate(cs[app_id]))
#Sort the list
sorted_scores = sorted(scores,key = lambda x:x[1],reverse = True)
sorted_scores = sorted_scores[1:]

In [39]:
steam_data_filtered['positive_ratings'] = steam_data_filtered['positive_ratings'].astype(str)

C:\Users\huang\AppData\Local\Temp\ipykernel_4260\2409004492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_data_filtered['positive_ratings'] = steam_data_filtered['positive_ratings'].astype(str)


In [40]:
#Print the first 10 recomended game
j = 0
print ('10 recomended game for',title,'are:\n')
for item in sorted_scores:
    game_title = (steam_data_filtered[steam_data_filtered.Game_no == item[0]]['name'].values[0]+' (Postitive Ratings : ' + steam_data_filtered[steam_data_filtered.Game_no == item[0]]['positive_ratings'].values[0] + ')')
    print(j+1,game_title)
    print('Similarity : ',end='')
    print(item[1])
    j = j+1
    if j>9 :
        break

10 recomended game for Counter-Strike are:

1 Counter-Strike: Source (Postitive Ratings : 76640)
Similarity : 0.9128709291752769
2 Counter-Strike: Condition Zero (Postitive Ratings : 12120)
Similarity : 0.8451542547285165
3 Counter-Strike: Global Offensive (Postitive Ratings : 2644404)
Similarity : 0.6761234037828132
4 Ricochet (Postitive Ratings : 2758)
Similarity : 0.6708203932499369
5 BRINK (Postitive Ratings : 3493)
Similarity : 0.6708203932499369
6 Homefront (Postitive Ratings : 5506)
Similarity : 0.6708203932499369
7 Blockstorm (Postitive Ratings : 5363)
Similarity : 0.6708203932499369
8 Battleborn (Postitive Ratings : 5692)
Similarity : 0.6708203932499369
9 Deathmatch Classic (Postitive Ratings : 1273)
Similarity : 0.6
10 Strike Vector (Postitive Ratings : 1560)
Similarity : 0.6


In [45]:

column_names = ['User_ID', 'Game_Name', 'Action', 'Hours', 'To drop']

user_data = pd.read_csv('../data/user.csv', header=None, names=column_names)
user_data.drop('To drop', axis=1, inplace=True)
user_data.head()

,User_ID,Game_Name,Action,Hours
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0
1,151603712,The Elder Scrolls V Skyrim,play,273.0
2,151603712,Fallout 4,purchase,1.0
3,151603712,Fallout 4,play,87.0
4,151603712,Spore,purchase,1.0


In [46]:
steam_data.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [47]:
filtered_user_data = user_data[user_data['Game_Name'].isin(steam_data['name'])]
filtered_user_data.describe()

,User_ID,Hours
count,9.282400e+04,92824.000000
mean,1.144695e+08,21.843521
std,7.586644e+07,168.810644
min,5.250000e+03,0.100000
25%,5.155740e+07,1.000000
50%,1.018789e+08,1.000000
75%,1.681638e+08,1.900000
max,3.099031e+08,10442.000000


In [57]:
filtered_user_data['Game_Name'].nunique()

2190

In [67]:
import re

# Function to clean the text by removing punctuations and extra spaces
def clean_text(text):
    # Remove punctuations
    text = re.sub(r'[^\w\s]', '', text)
    # Convert to lowercase and strip extra spaces
    return text.lower().strip()
games_df = steam_data.copy()
filtered_user_df = user_data.copy()

# Apply the cleaning function to the game titles in both dataframes
games_df['cleaned_title'] = games_df['name'].apply(clean_text)
filtered_user_df['cleaned_game_title'] = filtered_user_df['Game_Name'].apply(clean_text)

# Now filter the user data again based on the cleaned titles
filtered_user_df_cleaned = filtered_user_df[filtered_user_df['cleaned_game_title'].isin(games_df['cleaned_title'])]

# Display first few rows of the newly filtered user dataset
filtered_user_df_cleaned.head()


,User_ID,Game_Name,Action,Hours,cleaned_game_title
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,the elder scrolls v skyrim
1,151603712,The Elder Scrolls V Skyrim,play,273.0,the elder scrolls v skyrim
2,151603712,Fallout 4,purchase,1.0,fallout 4
3,151603712,Fallout 4,play,87.0,fallout 4
4,151603712,Spore,purchase,1.0,spore


In [77]:
filtered_user_df_cleaned['User_ID'].describe()

140553

In [79]:
games_df = games_df[games_df['cleaned_title'].isin(filtered_user_df_cleaned['cleaned_game_title'])]
games_df.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,cleaned_title
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19,counterstrike
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99,team fortress classic
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99,day of defeat
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99,deathmatch classic
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99,halflife opposing force


In [78]:
games_df.to_csv('../data/games_filtered.csv', index=False)
filtered_user_df_cleaned.to_csv('../data/users_filtered.csv', index=False)

In [81]:
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import svds
import numpy as np

users_filtered_df = filtered_user_df_cleaned.copy()
games_filtered_df = games_df.copy()

# Create a user-item matrix
user_ids = users_filtered_df['User_ID'].unique()
game_titles = games_filtered_df['cleaned_title'].unique()

# Map users and games to indices for matrix
user_map = {user: i for i, user in enumerate(user_ids)}
game_map = {game: i for i, game in enumerate(game_titles)}

# Convert playtime to ratings (more playtime, higher rating)
users_filtered_df['Rating'] = np.log(users_filtered_df['Hours'] + 1)

# Split the data into training and testing sets
train_df, test_df = train_test_split(users_filtered_df, test_size=0.2, random_state=42)

# Create a training matrix
train_matrix = coo_matrix((train_df['Rating'], 
                          (train_df['User_ID'].map(user_map), train_df['cleaned_game_title'].map(game_map))),
                          shape=(len(user_ids), len(game_titles)))

# Create a test matrix
test_matrix = coo_matrix((test_df['Rating'], 
                         (test_df['User_ID'].map(user_map), test_df['cleaned_game_title'].map(game_map))),
                         shape=(len(user_ids), len(game_titles)))

# Apply matrix factorization using SVD
U, sigma, Vt = svds(train_matrix, k=50)
sigma = np.diag(sigma)

# Predict ratings
predicted_ratings = np.dot(np.dot(U, sigma), Vt)

# Calculate RMSE on the test set
predicted_ratings_flattened = predicted_ratings[test_matrix.row, test_matrix.col]
rmse = np.sqrt(mean_squared_error(predicted_ratings_flattened, test_matrix.data))

rmse

1.7559996854619309

In [82]:
# Number of games to recommend
num_recommendations = 10

# Get recommendations for the first few users
recommendations = {}
for user_idx, user in enumerate(user_ids[:5]):
    # Get predicted ratings for the user
    user_ratings = predicted_ratings[user_idx]
    
    # Get games the user has already interacted with
    known_games = users_filtered_df[users_filtered_df['User_ID'] == user]['cleaned_game_title'].tolist()
    
    # Exclude already known games from recommendations
    user_ratings = [(rating, game) for game, rating in enumerate(user_ratings) if game_titles[game] not in known_games]
    
    # Sort the games based on predicted ratings and get top recommendations
    sorted_ratings = sorted(user_ratings, key=lambda x: x[0], reverse=True)
    recommended_games = [game_titles[r[1]] for r in sorted_ratings[:num_recommendations]]
    
    # Store recommendations for the user
    recommendations[user] = recommended_games

recommendations

{151603712: ['the witcher 2 assassins of kings enhanced edition',
  'dishonored',
  'rage',
  'the witcher 3 wild hunt',
  'adventure capitalist',
  'arma 2 operation arrowhead',
  'killing floor',
  'torchlight ii',
  'neverwinter',
  'dying light'],
 187131847: ['gunz 2 the second duel',
  'neverwinter',
  'aura kingdom',
  'archeblade',
  'quake live',
  'smite',
  'toribash',
  'stronghold kingdoms',
  'spiral knights',
  'gotham city impostors free to play'],
 59945701: ['warhammer 40000 dawn of war ii',
  'left 4 dead',
  'killing floor 2',
  'grand theft auto iv',
  'saints row the third',
  'dark souls ii',
  'alien swarm',
  'dungeon defenders',
  'xcom enemy unknown',
  'path of exile'],
 53875128: ['heroes  generals',
  'starbound',
  'chivalry medieval warfare',
  'sid meiers civilization iv',
  'assassins creed revelations',
  'batman arkham origins',
  'arma 3',
  'grand theft auto episodes from liberty city',
  'assassins creed brotherhood',
  'worms revolution'],
 23494

In [83]:
# Number of games to recommend
num_recommendations = 10

# Create a dictionary to store recommendations for each user
recommendations = {}

for user_idx, user in enumerate(user_ids[:5]):  # Here, we're generating recommendations for the first 5 users as an example
    # Get predicted ratings for the user
    user_ratings = predicted_ratings[user_idx]
    
    # Get games the user has already interacted with
    known_games = users_filtered_df[users_filtered_df['User_ID'] == user]['cleaned_game_title'].tolist()
    
    # Exclude already known games from recommendations
    user_ratings = [(rating, game) for game, rating in enumerate(user_ratings) if game_titles[game] not in known_games]
    
    # Sort the games based on predicted ratings and get top recommendations
    sorted_ratings = sorted(user_ratings, key=lambda x: x[0], reverse=True)
    recommended_games = [game_titles[r[1]] for r in sorted_ratings[:num_recommendations]]
    
    # Store recommendations for the user
    recommendations[user] = recommended_games

# Print recommendations for the users
for user, games in recommendations.items():
    print(f"Recommendations for user {user}:")
    for game in games:
        print(f"- {game}")
    print("\n")


Recommendations for user 151603712:
- the witcher 2 assassins of kings enhanced edition
- dishonored
- rage
- the witcher 3 wild hunt
- adventure capitalist
- arma 2 operation arrowhead
- killing floor
- torchlight ii
- neverwinter
- dying light


Recommendations for user 187131847:
- gunz 2 the second duel
- neverwinter
- aura kingdom
- archeblade
- quake live
- smite
- toribash
- stronghold kingdoms
- spiral knights
- gotham city impostors free to play


Recommendations for user 59945701:
- warhammer 40000 dawn of war ii
- left 4 dead
- killing floor 2
- grand theft auto iv
- saints row the third
- dark souls ii
- alien swarm
- dungeon defenders
- xcom enemy unknown
- path of exile


Recommendations for user 53875128:
- heroes  generals
- starbound
- chivalry medieval warfare
- sid meiers civilization iv
- assassins creed revelations
- batman arkham origins
- arma 3
- grand theft auto episodes from liberty city
- assassins creed brotherhood
- worms revolution


Recommendations for us

In [85]:
# new model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Combine relevant columns to create a content feature for each game
games_filtered_df['content'] = games_filtered_df['genres'] + " " + games_filtered_df['steamspy_tags']

# Use TF-IDF to vectorize the content
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(games_filtered_df['content'])

tfidf_matrix.shape

(3159, 296)

In [88]:
# 1. 特征向量化
# 使用TF-IDF对游戏的内容进行向量化
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(games_filtered_df['content'])

# 2. 计算用户兴趣向量
def get_user_interest_vector(user_games):
    # 获取用户已经玩过的游戏的索引
    game_indices = games_filtered_df[games_filtered_df['cleaned_title'].isin(user_games)].index
    # 计算这些游戏的特征向量的平均值
    user_vector = tfidf_matrix[game_indices].mean(axis=0)
    return user_vector

# 为了示例，假设用户玩过的游戏是以下列表：
user_played_games = ["counterstrike", "team fortress classic"]
user_vector = get_user_interest_vector(user_played_games)

# 3. 计算相似度
cosine_similarities = cosine_similarity(np.asarray(user_vector), tfidf_matrix)

# 4. 排序和推荐
recommended_game_indices = cosine_similarities.argsort().flatten()[-5:]
recommended_games = games_filtered_df.iloc[recommended_game_indices]['cleaned_title'].tolist()

print("Recommended games for the user:", recommended_games)


Recommended games for the user: ['call of duty 4 modern warfare', 'shootmania storm', 'halflife 2 deathmatch', 'homefront', 'counterstrike']


In [12]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.neighbors import NearestNeighbors

# Load Data
games_filtered_df = pd.read_csv("../data/games_filtered.csv")
users_df = pd.read_csv("../data/users_filtered.csv")

# Transformer for the 'steamspy_tags' feature
class GameInfoTransformerTFIDF:        
    def __init__(self):
        self.vectorizer = TfidfVectorizer(stop_words='english')
    
    def fit(self, X, y=None):
        self.vectorizer.fit(X['steamspy_tags'])
        return self
    
    def transform(self, X):
        return self.vectorizer.transform(X['steamspy_tags']).toarray()

# Transformer for 'genres'
class GenresTransformer:    
    def __init__(self):
        self.mlb = MultiLabelBinarizer()
        
    def fit(self, X, y=None):
        self.mlb.fit(X['genres'].str.split(';'))
        return self
    
    def transform(self, X):
        return self.mlb.transform(X['genres'].str.split(';'))

# FeatureUnion to combine features
gameFeatureTransformer = FeatureUnion([
    ('info_processing', Pipeline([('info', GameInfoTransformerTFIDF())])),
    ('genres_processing', Pipeline([('genres', GenresTransformer())]))
])

# featureTransformer with pca
gameFeatureTransformer_pca = Pipeline([
    ('features', gameFeatureTransformer),
    ('pca', PCA(n_components=100))  # Reducing dimensions to 100
])

# Fit and transform game data
gameFeatureTransformer_pca.fit(games_filtered_df)
transformed_games_pca = gameFeatureTransformer_pca.transform(games_filtered_df)

# Train NearestNeighbors model
K = 10
game_model_pca = NearestNeighbors(algorithm="brute", n_neighbors=K+1)
game_model_pca.fit(transformed_games_pca)

# Recommendation function
def recommend_games_pca(model, gamesTaken, k=5):
    aggregate_vector = np.zeros((1, 100))
    for game in gamesTaken:
        game_data = games_filtered_df[games_filtered_df['cleaned_title'] == game]
        if not game_data.empty:
            game_vector = gameFeatureTransformer_pca.transform(game_data)
            aggregate_vector += game_vector

    distances, indices = model.kneighbors(aggregate_vector, n_neighbors=k+1)
    recommendations = [games_filtered_df.iloc[index]['cleaned_title'] for index in indices[0]][1:]
    return recommendations

# Example usage:
sample_games = ["Counter-Strike", "Team Fortress 2"]
recommended_games = recommend_games_pca(game_model_pca, sample_games, 5)
print(recommended_games)


['over the void', 'screencheat', 'boson x', 'korwin the game', 'tank universal']


In [33]:
def compute_precision_recall(recommended, ground_truth):
    # True Positives
    TP = len(set(recommended).intersection(set(ground_truth)))
    # False Positives
    FP = len(set(recommended) - set(ground_truth))
    # False Negatives
    FN = len(set(ground_truth) - set(recommended))
    
    # Precision and Recall
    precision = TP / (TP + FP) if TP + FP != 0 else 0
    recall = TP / (TP + FN) if TP + FN != 0 else 0
    
    return precision, recall


In [17]:
games_filtered_df.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,cleaned_title
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19,counterstrike
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99,team fortress classic
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99,day of defeat
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99,deathmatch classic
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99,halflife opposing force


In [22]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from datetime import datetime

class CategoriesGenresTagsTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.vectorizer = TfidfVectorizer(stop_words='english')
        
    def fit(self, X, y=None):
        combined_text = X['categories'] + " " + X['genres'] + " " + X['steamspy_tags']
        self.vectorizer.fit(combined_text)
        return self
    
    def transform(self, X):
        combined_text = X['categories'] + " " + X['genres'] + " " + X['steamspy_tags']
        return self.vectorizer.transform(combined_text).toarray()

class RatingsTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        total_ratings = X['positive_ratings'] + X['negative_ratings']
        rating_ratio = X['positive_ratings'] / total_ratings
        return rating_ratio.values.reshape(-1, 1)

class PlaytimeTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.average_playtime_max = X['average_playtime'].max()
        self.median_playtime_max = X['median_playtime'].max()
        return self
    
    def transform(self, X):
        normalized_average_playtime = X['average_playtime'] / self.average_playtime_max
        normalized_median_playtime = X['median_playtime'] / self.median_playtime_max
        return pd.concat([normalized_average_playtime, normalized_median_playtime], axis=1).values

class OwnersTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        owners_processed = X['owners'].str.split('-').apply(lambda x: (int(x[0]) + int(x[1])) / 2)
        return owners_processed.values.reshape(-1, 1)

class DaysSinceReleaseTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        now = datetime.now()
        days_since_release = X['release_date'].apply(lambda x: (now - datetime.strptime(x, '%Y-%m-%d')).days)
        return days_since_release.values.reshape(-1, 1)

# Combined Feature Transformer
feature_transformer = FeatureUnion([
    ('categories_genres_tags', CategoriesGenresTagsTransformer()),
    ('ratings', RatingsTransformer()),
    ('playtime', PlaytimeTransformer()),
    ('owners', OwnersTransformer()),
    ('days_since_release', DaysSinceReleaseTransformer())
])


In [31]:
from sklearn.metrics.pairwise import cosine_similarity

# 加载数据
games_df = pd.read_csv("../data/games_filtered.csv")
users_df = pd.read_csv("../data/users_filtered.csv")

# 处理数据
users_df['Hours'] = np.where(users_df['Action'] == 'purchase', 0, users_df['Hours'])

# 使用feature_transformer计算每款游戏的特征向量

feature_transformer.fit(games_df)
game_features = feature_transformer.transform(games_df)

# 将游戏标题与其对应的特征向量一一对应起来，形成一个字典
game_features_dict = {game: features for game, features in zip(games_df['cleaned_title'], game_features)}

# 基于用户玩的游戏和游玩时长计算用户表示
def compute_user_representation(user_id, users_df, game_features_dict):
    user_games = users_df[users_df['User_ID'] == user_id]
    representation = np.zeros(next(iter(game_features_dict.values())).shape)
    for _, row in user_games.iterrows():
        game_title = row['cleaned_game_title']
        hours = row['Hours']
        representation += game_features_dict[game_title] * hours
    return representation / len(user_games)

# 找到与目标用户最接近的游戏
def recommend_games_based_on_user_representation(user_id, game_features, top_n=5):
    user_representation = compute_user_representation(user_id, users_df, game_features_dict)
    similarities = cosine_similarity([user_representation], game_features)
    recommended_indices = similarities[0].argsort()[-top_n:][::-1]
    recommended_games = games_df.iloc[recommended_indices]['cleaned_title'].tolist()
    return recommended_games

# 示例
user_id_example = users_df['User_ID'].iloc[0]
print(recommend_games_based_on_user_representation(user_id_example, game_features))


['left 4 dead 2', 'portal 2', 'ark survival evolved', 'the elder scrolls v skyrim', 'tom clancys rainbow six siege']


In [39]:
from sklearn.metrics.pairwise import cosine_similarity

def evaluate_using_cosine_similarity(user_id, users_df, game_features_dict):
    user_representation = compute_user_representation(user_id, users_df, game_features_dict)
    
    known_games = users_df[users_df['User_ID'] == user_id]['cleaned_game_title'].tolist()
    known_games_rep = np.mean([game_features_dict[game] for game in known_games], axis=0)
    
    similarity = cosine_similarity([user_representation], [known_games_rep])
    return similarity[0][0]

all_similarities = []

for user_id in users_df['User_ID'].unique():
    similarity = evaluate_using_cosine_similarity(user_id, users_df, game_features_dict)
    all_similarities.append(similarity)

avg_similarity = np.mean(all_similarities)

print(f"Average Cosine Similarity: {avg_similarity}")


Average Cosine Similarity: 0.9225307326446769


In [38]:
users_df.head()

,User_ID,Game_Name,Action,Hours,cleaned_game_title
0,151603712,The Elder Scrolls V Skyrim,purchase,0.0,the elder scrolls v skyrim
1,151603712,The Elder Scrolls V Skyrim,play,273.0,the elder scrolls v skyrim
2,151603712,Fallout 4,purchase,0.0,fallout 4
3,151603712,Fallout 4,play,87.0,fallout 4
4,151603712,Spore,purchase,0.0,spore
